**Column Details:**
- **State_Name**: Name of the Indian state where the crop was grown.  
- **District_Name**: Name of the district within the state.  
- **Crop_Year**: Year in which the crop was cultivated.  
- **Season**: Growing season of the crop (e.g., Kharif, Rabi, etc.).  
- **Crop**: Name of the crop grown (e.g., rice, wheat, maize, etc.).  
- **Area**: Total area (in hectares) under cultivation for the crop.  
- **Production**: Total production (in metric tons) of the crop in that district/year.  
